# Talking to CSV and Excel files with LangChain

In [21]:
!pip -q install langchain openai

In [45]:
import os

os.environ["OPENAI_API_KEY"] = ""

In [23]:
!pip show langchain

Name: langchain
Version: 0.0.337
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: c:\users\vedan\.conda\envs\long_charin\lib\site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [24]:
import pandas as pd

df = pd.read_csv('bigBasketProducts.csv')

In [25]:
df.head()

,index,product,category,sub_category,brand,sale_price,market_price,type,rating,description
0,1,Garlic Oil - Vegetarian Capsule 500 mg,Beauty & Hygiene,Hair Care,Sri Sri Ayurveda,220.0,220.0,Hair Oil & Serum,4.1,This Product contains Garlic Oil that is known...
1,2,Water Bottle - Orange,"Kitchen, Garden & Pets",Storage & Accessories,Mastercook,180.0,180.0,Water & Fridge Bottles,2.3,"Each product is microwave safe (without lid), ..."
2,3,"Brass Angle Deep - Plain, No.2",Cleaning & Household,Pooja Needs,Trm,119.0,250.0,Lamp & Lamp Oil,3.4,"A perfect gift for all occasions, be it your m..."
3,4,Cereal Flip Lid Container/Storage Jar - Assort...,Cleaning & Household,Bins & Bathroom Ware,Nakoda,149.0,176.0,"Laundry, Storage Baskets",3.7,Multipurpose container with an attractive desi...
4,5,Creme Soft Soap - For Hands & Body,Beauty & Hygiene,Bath & Hand Wash,Nivea,162.0,162.0,Bathing Bars & Soaps,4.4,Nivea Creme Soft Soap gives your skin the best...


In [29]:
!pip install langchain-experimental


     -------------------------------------- 157.0/157.0 kB 1.2 MB/s eta 0:00:00


In [33]:
from langchain_experimental.agents.agent_toolkits import create_csv_agent
from langchain.llms import OpenAI

In [34]:
agent = create_csv_agent(OpenAI(temperature=0),
                         'bigBasketProducts.csv',
                         verbose=True)

In [35]:
agent

AgentExecutor(verbose=True, agent=ZeroShotAgent(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'input'], partial_variables={'df_head': '|    |   index | product                                                 | category               | sub_category          | brand            |   sale_price |   market_price | type                     |   rating | description                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

In [36]:
agent.agent.llm_chain.prompt.template

'\nYou are working with a pandas dataframe in Python. The name of the dataframe is `df`.\nYou should use the tools below to answer the question posed of you:\n\npython_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [python_repl_ast]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\n\nThis is the result of `print(df.head())`:\n{df_head}\n\nBegin!\nQuestion: {input}\n{agent_scratchpad}'


You are working with a pandas dataframe in Python. The name of the dataframe is `df`.
You should use the tools below to answer the question posed of you:

python_repl_ast: A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [python_repl_ast]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question


This is the result of `print(df.head())`:
{df}

Begin!
Question: {input}
{agent_scratchpad}

In [41]:
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain_experimental.agents.agent_toolkits import create_csv_agent

In [46]:
agent = create_csv_agent(
    OpenAI(temperature=0),
    "bigBasketProducts.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [44]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
agent.run("how many people are female?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of people who are female
Action: python_repl_ast
Action Input: df[df['Gender'] == 'F'].count()
Observation: User_ID                       135809
Product_ID                    135809
Gender                        135809
Age                           135809
Occupation                    135809
City_Category                 135809
Stay_In_Current_City_Years    135809
Marital_Status                135809
Product_Category_1            135809
Product_Category_2             91530
Product_Category_3             37594
Purchase                      135809
dtype: int64
Thought: I now know the final answer
Final Answer: 135809

> Finished chain.


'135809'

In [ ]:
agent.run("how many people have stayed more than 3 years in the city?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to find the people who have stayed more than 3 years in the city
Action: python_repl_ast
Action Input: df[df['Stay_In_Current_City_Years'] > '3'].shape[0]
Observation: 84726
Thought: I now know the final answer
Final Answer: 84726 people have stayed more than 3 years in the city.

> Finished chain.


'84726 people have stayed more than 3 years in the city.'

In [ ]:
agent.run("how many people have stayed more than 3 years in the city and are female?")



> Entering new AgentExecutor chain...
Thought: I need to filter the dataframe to find the people who meet the criteria
Action: python_repl_ast
Action Input: df[(df['Stay_In_Current_City_Years'] > '3') & (df['Gender'] == 'F')]
Observation:         User_ID Product_ID Gender    Age  Occupation City_Category  \
29      1000010  P00085942      F  36-45           1             B   
30      1000010  P00118742      F  36-45           1             B   
31      1000010  P00297942      F  36-45           1             B   
32      1000010  P00266842      F  36-45           1             B   
33      1000010  P00058342      F  36-45           1             B   
...         ...        ...    ...    ...         ...           ...   
549972  1005901  P00372445      F  36-45           7             C   
549981  1005911  P00372445      F  26-35          17             C   
550011  1005952  P00371644      F  46-50           1             C   
550065  1006036  P00375436      F  26-35          15       

'18596 people have stayed more than 3 years in the city and are female.'

In [ ]:
agent.run("Are there more males or females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females
Action: python_repl_ast
Action Input: df['Gender'].value_counts()
Observation: M    414259
F    135809
Name: Gender, dtype: int64
Thought: I now know the final answer
Final Answer: There are more males than females.

> Finished chain.


'There are more males than females.'

## LangChain CSV Loader

In [ ]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/train.csv')
data = loader.load()

In [ ]:
#custom chain etc

## Convert XLS file to csv

In [ ]:
!pip install openpyxl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!wget -q https://www.dropbox.com/s/co5kwipba9hpumt/excel_files.zip
!unzip -q excel_files.zip

In [ ]:
xls_file = r'/content/excel_file_example.xlsx'
output_csv = r'/content/excel_file_example.csv'

# Read the XLS file using pandas and openpyxl as the engine
data = pd.read_excel(xls_file, engine='openpyxl')

# Save the data as a CSV file
data.to_csv(output_csv, index=False)

In [ ]:
agent = create_csv_agent(OpenAI(temperature=0),
                         '/content/excel_file_example.csv',
                         verbose=True)

In [ ]:
agent.run("What are the column names?")



> Entering new AgentExecutor chain...
Thought: I need to find out what the column names are
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['0', 'First Name', 'Last Name', 'Gender', 'Country', 'Age', 'Date',
       'Id'],
      dtype='object')
Thought: I now know the column names
Final Answer: 0, First Name, Last Name, Gender, Country, Age, Date, Id

> Finished chain.


'0, First Name, Last Name, Gender, Country, Age, Date, Id'

In [ ]:
agent.run("What is the average age?")



> Entering new AgentExecutor chain...
Thought: I need to calculate the average age of the people in the dataframe
Action: python_repl_ast
Action Input: df['Age'].mean()
Observation: 33.26
Thought: I now know the final answer
Final Answer: The average age is 33.26.

> Finished chain.


'The average age is 33.26.'

In [ ]:
agent.run("Which country appears the most and how many times does it appear?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of times each country appears in the dataframe.
Action: python_repl_ast
Action Input: df['Country'].value_counts()
Observation: United States    48
Great Britain    28
France           24
Name: Country, dtype: int64
Thought: I now know the final answer
Final Answer: The country that appears the most is United States, appearing 48 times.

> Finished chain.


'The country that appears the most is United States, appearing 48 times.'

In [ ]:
agent.run("What is the ratio of males to females?")



> Entering new AgentExecutor chain...
Thought: I need to count the number of males and females
Action: python_repl_ast
Action Input: df['Gender'].value_counts()
Observation: Female    76
Male      24
Name: Gender, dtype: int64
Thought: I now know the final answer
Final Answer: 24:76

> Finished chain.


'24:76'